In [1]:
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

     |████████████████████████████████| 58 kB 3.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72857 sha256=fd15f57607bc5ecae9470a4f91a110b361e033deecce4931cd19459f3a4cbb30
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [2]:
!kaggle competitions download -c 2021-ml-midterm-p3

  0% 0.00/68.5k [00:00<?, ?B/s]
100% 68.5k/68.5k [00:00<00:00, 35.4MB/s]


In [3]:
!unzip 2021-ml-midterm-p3.zip

Archive:  2021-ml-midterm-p3.zip
  inflating: sample_submit.csv       
  inflating: test.csv                
  inflating: train.csv               


In [5]:
import pandas as pd
import numpy as np

train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
submit=pd.read_csv("sample_submit.csv")

In [9]:
x=train.drop(["index","label"],axis=1)
y=train["label"]
test=test.drop("index",axis=1)

In [13]:
import torch

seed=777

torch.manual_seed(seed)

device='cuda' if torch.cuda.is_available() else 'cpu'

if device=='cuda':
  torch.cuda.manual_seed_all(seed)

In [60]:
linear1=torch.nn.Linear(64,128)
linear2=torch.nn.Linear(128,256)
linear3=torch.nn.Linear(256,128)
linear4=torch.nn.Linear(128,64)
linear5=torch.nn.Linear(64,10)

relu=torch.nn.ReLU()

dropout=torch.nn.Dropout(p=0.2)

In [61]:
torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)
torch.nn.init.xavier_normal_(linear4.weight)
torch.nn.init.xavier_normal_(linear5.weight)


model=torch.nn.Sequential(linear1,relu,dropout,linear2,relu,dropout,linear3,relu,dropout,linear4,relu,dropout,linear5).to(device)

In [62]:
optim=torch.optim.Adam(params=model.parameters(),lr=0.001)

loss=torch.nn.CrossEntropyLoss().to(device)

In [63]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x)
x_std=scaler.transform(x)
test_std=scaler.transform(test)

In [64]:
x=torch.FloatTensor(np.array(x_std))
y=torch.LongTensor(np.array(y))
test=torch.FloatTensor(np.array(test_std))

In [65]:
model.train()

for stop in range(600):
  optim.zero_grad()

  h=model(x.to(device))

  cost=loss(h,y.to(device))

  cost.backward()

  optim.step()

  if stop % 100==0:
    print(stop,cost.item())

0 2.378664016723633
100 0.035244811326265335
200 0.0105561139062047
300 0.0025651275645941496
400 0.0012065964983776212
500 0.0033589007798582315


In [66]:
pred=list()

with torch.no_grad():
  model.eval()
  test_y=model(test.to(device))
  a,b=torch.max(test_y,dim=1)
  pred.extend(b.tolist())

In [67]:
submit["label"]=pred

submit.to_csv("submit.csv",index=False)

In [68]:
!kaggle competitions submit -c 2021-ml-midterm-p3 -f submit.csv -m "message"

100% 2.26k/2.26k [00:03<00:00, 580B/s]
Successfully submitted to 2021.ML.MidTerm.P3